In [1]:
import googlemaps
#import unicodedata
import pandas as pd

In [2]:
#Local xls file with postal codes
#Rename the xls sheets
xls_file = pd.ExcelFile('C:\\Users\\thierry\\Documents\\DATA\\GEO coding BE\\zipcodes_prov_fr.xls')

In [3]:
capitalizer = lambda x: x.upper()

In [4]:
#Replace with you API key
gmaps = googlemaps.Client(key='')

In [5]:
table_all = xls_file.parse('BW')
table_all['sheet'] = 'BW'
table_all['lang'] = 'FR'

In [6]:
table_sheet = xls_file.parse('BXL')
table_sheet['sheet'] = 'BXL'
table_sheet['lang'] = 'FR'
table_all = table_all.append(table_sheet,ignore_index=True)

In [7]:
table_sheet = xls_file.parse('VB')
table_sheet['sheet'] = 'VB'
table_sheet['lang'] = 'NL'
table_all = table_all.append(table_sheet,ignore_index=True)

In [8]:
table_sheet = xls_file.parse('AN')
table_sheet['sheet'] = 'AN'
table_sheet['lang'] = 'NL'
table_all = table_all.append(table_sheet,ignore_index=True)

In [9]:
table_sheet = xls_file.parse('LI')
table_sheet['sheet'] = 'LI'
table_sheet['lang'] = 'NL'
table_all = table_all.append(table_sheet,ignore_index=True)

In [10]:
table_sheet = xls_file.parse('LG')
table_sheet['sheet'] = 'LG'
table_sheet['lang'] = 'NL'
table_all = table_all.append(table_sheet,ignore_index=True)

In [11]:
table_sheet = xls_file.parse('NA')
table_sheet['sheet'] = 'NA'
table_sheet['lang'] = 'FR'
table_all = table_all.append(table_sheet,ignore_index=True)

In [12]:
table_sheet = xls_file.parse('HT')
table_sheet['sheet'] = 'HT'
table_sheet['lang'] = 'FR'
table_all = table_all.append(table_sheet,ignore_index=True)

In [13]:
table_sheet = xls_file.parse('LX')
table_sheet['sheet'] = 'LX'
table_sheet['lang'] = 'FR'
table_all = table_all.append(table_sheet,ignore_index=True)

In [14]:
table_sheet = xls_file.parse('WV')
table_sheet['sheet'] = 'WV'
table_sheet['lang'] = 'NL'
table_all = table_all.append(table_sheet,ignore_index=True)

In [15]:
table_sheet = xls_file.parse('OV')
table_sheet['sheet'] = 'OV'
table_sheet['lang'] = 'NL'
table_all = table_all.append(table_sheet,ignore_index=True)

In [16]:
table_all['Sous-commune'] = table_all['Sous-commune'].map(capitalizer)

In [17]:
table_all.head(n=5)

,Code postal,Localité,Commune,Sous-commune,Rôle linguistique,sheet,lang
0,1300,Limal,WAVRE,OUI,F,BW,FR
1,1300,WAVRE,WAVRE,NON,F,BW,FR
2,1301,Bierges,WAVRE,OUI,F,BW,FR
3,1310,LA HULPE,LA HULPE,NON,F,BW,FR
4,1315,Glimes,INCOURT,OUI,F,BW,FR


In [18]:
table_all.tail(n=5)

,Code postal,Localité,Commune,Sous-commune,Rôle linguistique,sheet,lang
2771,9988,Waterland-Oudeman,SINT-LAUREINS,OUI,N,OV,NL
2772,9988,Watervliet,SINT-LAUREINS,OUI,N,OV,NL
2773,9990,MALDEGEM,MALDEGEM,NON,N,OV,NL
2774,9991,Adegem,MALDEGEM,OUI,N,OV,NL
2775,9992,Middelburg,MALDEGEM,OUI,N,OV,NL


## Iterate over file for each line get, lat & lng

In [19]:
def get_city_geocode(in_postal_city,in_language):
    try:
      geocode_result = gmaps.geocode(in_postal_city,language=in_language,region="BE")
      for address_components in geocode_result[0]['address_components']:
        if address_components['types'][0] == 'country' and address_components['short_name'] == 'BE':            
          return get_geo_info(geocode_result)      
          break
    except:
      print "  Exception"

In [20]:
def get_geo_info(in_geocode_result):
  geo_result = {}
  for address_components in in_geocode_result[0]['address_components']:
    if address_components['types'][0] == 'country':
      geo_result['country'] = address_components['short_name']
    if address_components['types'][0] == 'postal_code': 
      geo_result['postal_code'] = address_components['short_name']        
    if address_components['types'][0] == 'locality': 
      geo_result['locality'] = address_components['short_name']        
    if address_components['types'][0] == 'political': 
      geo_result['political'] = address_components['short_name']        
    if address_components['types'][0] == 'administrative_area_level_1':
      geo_result['administrative_area_level_1'] = address_components['short_name']        
    if address_components['types'][0] == 'administrative_area_level_2':   
      geo_result['administrative_area_level_2'] = address_components['short_name']
    geo_result['place_id'] = in_geocode_result[0]['place_id']
    geo_result['formatted_address'] = in_geocode_result[0]['formatted_address']
    geo_result['lat'] = in_geocode_result[0]['geometry']['location']['lat']
    geo_result['lng'] = in_geocode_result[0]['geometry']['location']['lng']
  return geo_result
#    print geo_result  

In [21]:
geo_result = []
for xls_row in table_all.itertuples():
    if str(xls_row[4]) == "OUI":
      city_be = (str(xls_row[1]), xls_row[2], 'Belgique')
    else:
      city_be = (str(xls_row[1]), 'Belgique')
    city_be = " ".join(city_be)
    print city_be
    geocoded = get_city_geocode(city_be,str(xls_row[7]))
    if isinstance(geocoded, dict):
      if not 'locality' in geocoded:
        geocoded['locality'] = xls_row[2]
      geo_result.append(geocoded)
        
geo_result_frame = pd.DataFrame(geo_result)

1300 Limal Belgique
1300 Belgique
1301 Bierges Belgique
1310 Belgique
1315 Glimes Belgique
1315 Belgique
1315 Opprebais Belgique
1315 Piètrebais Belgique
1315 Roux-Miroir Belgique
1320 Belgique
1320 Hamme-Mille Belgique
1320 L'Ecluse Belgique
1320 Nodebais Belgique
1320 Tourinnes-La-Grosse Belgique
1325 Bonlez Belgique
1325 Belgique
1325 Corroy-Le-Grand Belgique
1325 Longueville Belgique
1330 Belgique
1331 Rosières Belgique
1332 Genval Belgique
1340 Ottignies Belgique
1340 Belgique
1341 Céroux-Mousty Belgique
1342 Limelette Belgique
1348 Louvain-La-Neuve Belgique
1350 Enines Belgique
1350 Folx-Les-Caves Belgique
1350 Jandrain-Jandrenouille Belgique
1350 Jauche Belgique
1350 Marilles Belgique
1350 Noduwez Belgique
1350 Belgique
1350 Orp-Le-Grand Belgique
1357 Belgique
1357 Linsmeau Belgique
1357 Neerheylissem Belgique
1357 Opheylissem Belgique
1360 Malèves-Sainte-Marie-Wastines Belgique
1360 Orbais Belgique
1360 Belgique
1360 Thorembais-Les-Béguines Belgique
1360 Thorembais-Saint-Trond 

In [22]:
#Save the result to local file
writer = pd.ExcelWriter('C:\\Users\\thierry\\Documents\\DATA\\GEO coding BE\\zipcodes_prov_fr_ALL.xlsx')
geo_result_frame.to_excel(writer,'ALL')
writer.save()

#### Help functions

In [ ]:
for xls_row in table_all.itertuples():
        print xls_row[1], xls_row[2], xls_row[3], xls_row[4], xls_row[5], xls_row[6], xls_row[7]
    

In [ ]:
def get_geo_info_print(in_geocode_result):
  for address_components in in_geocode_result[0]['address_components']:
    if address_components['types'][0] == 'country': 
      print "  Country: " + address_components['short_name']
    if address_components['types'][0] == 'postal_code': 
      print "  Postal code: " + address_components['short_name']
    if address_components['types'][0] == 'locality': 
      print "  locality: " + address_components['short_name']
    if address_components['types'][0] == 'political': 
      print "  political: " + address_components['short_name']
    if address_components['types'][0] == 'administrative_area_level_1':
      print "  admin level 1: " + address_components['short_name']
    if address_components['types'][0] == 'administrative_area_level_2':        
      print "  admin level 2: " + address_components['short_name']    
  print "  Place ID: " + in_geocode_result[0]['place_id'] 
  print "  Formatted address: " + in_geocode_result[0]['formatted_address']
  print "  lat: "  + str(in_geocode_result[0]['geometry']['location']['lat'])
  print "  lng: "  + str(in_geocode_result[0]['geometry']['location']['lng'])    